# Module 2. Deploy Multi-container Endpoint
---

## Overview

SageMaker 멀티 컨테이너 엔드포인트를 사용하면 서로 다른 serving 스택(예: 모델 서버, 머신 러닝 프레임워크, 프레임워크 버전, 알고리즘 등)에 구축된 여러 추론 컨테이너를 하나의 엔드포인트에서 실행하고 독립적으로 각 추론 컨테이너를 호출할 수 있습니다. 

- 인스턴스의 전체 수용량을 포화시킬 정도의 트래픽이 없는 경우에 여러 모델(예: Object Detection, Named Entity Recognition)을 서빙
- A/B 테스트와 같은 시나리오에서 서로 다른 프레임워크 버전(예: TensorFlow 1.x vs. TensorFlow 2.x)에서 실행되는 유사한 아키텍처의 비교

<br>

## 1. Upload Model Artifacts
---

모델을 아카이빙하여 S3로 업로드합니다.

In [ ]:
!pip install transformers==4.11.3

In [ ]:
import torch
import torchvision
import torchvision.models as models
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
import boto3
import datetime
import time
from time import strftime,gmtime
import json
import os
import io
import torchvision.transforms as transforms
from src.utils import print_outputs, upload_model_artifact_to_s3, NLPPredictor 

role = get_execution_role()
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")
region = boto_session.region_name
bucket = sm_session.default_bucket()
prefix = 'multi-container-nlp'

print(f'region = {region}')
print(f'role = {role}')
print(f'bucket = {bucket}')
print(f'prefix = {prefix}')

In [ ]:
modelA_variant = 'modelA'
modelA_path = 'model-nsmc'
modelA_s3_uri = upload_model_artifact_to_s3(modelA_variant, modelA_path, bucket, prefix)

In [ ]:
modelB_variant = 'modelB'
modelB_path = 'model-korsts'
modelB_s3_uri = upload_model_artifact_to_s3(modelB_variant, modelB_path, bucket, prefix)

In [ ]:
modelC_variant = 'modelC'
modelC_path = 'model-kobart'
modelC_s3_uri = upload_model_artifact_to_s3(modelC_variant, modelC_path, bucket, prefix)

<br>

## 2. Create Multi-container endpoint
---

SageMaker 멀티 컨테이너 엔드포인트를 사용하면 여러 컨테이너들을 동일한 엔드포인트에 배포할 수 있으며, 각 컨테이너에 개별적으로 액세스하여 비용을 최적화할 수 있습니다. 다중 컨테이너 엔드포인트 설정은 기존 엔드포인트와 유사하지만, SageMaker 모델 생성 시 여러 컨테이너들을 명시해 줘야 합니다.

- 배포에 필요한 각 컨테이너에 대한 추론 컨테이너 정의 생성
- `create_model` API를 사용하여 SageMaker 모델 생성; PrimaryContainer 대신 Containers 매개변수를 사용하고 Containers 매개변수에 두 개 이상의 컨테이너를 포함합니다. (최대 15개까지 지원)
- `create_endpoint_config` API를 사용하여 SageMaker 엔드포인트 설정 생성
- `create_endpoint` API를 사용하여 SageMaker 엔드포인트 생성; 생성 시 반드시 Direct 모드를 사용해야 합니다.

자세한 내용은 아래 링크를 참조해 주세요.
- Host Multiple Models with Multi-Model Endpoints: https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html

### Create Inference containter definition for Model A

In [ ]:
from sagemaker.image_uris import retrieve

deploy_instance_type = 'ml.c5.xlarge'
pt_ecr_image_uriA = retrieve(
    framework='pytorch',
    region=region,
    version='1.7.1',
    py_version='py3',
    instance_type = deploy_instance_type,
    accelerator_type=None,
    image_scope='inference'
)

pt_containerA = {
    "ContainerHostname": "pytorch-kornlp-nsmc",
    "Image": pt_ecr_image_uriA,
    "ModelDataUrl": modelA_s3_uri,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference_nsmc.py",
        "SAGEMAKER_SUBMIT_DIRECTORY": modelA_s3_uri,
    },
}

### Create Inference containter definition for Model B

In [ ]:
pt_ecr_image_uriB = retrieve(
    framework='pytorch',
    region=region,
    version='1.8.1',
    py_version='py3',
    instance_type = deploy_instance_type,
    accelerator_type=None,
    image_scope='inference'
)

pt_containerB = {
    "ContainerHostname": "pytorch-kornlp-korsts",
    "Image": pt_ecr_image_uriB,
    "ModelDataUrl": modelB_s3_uri,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference_korsts.py",
        "SAGEMAKER_SUBMIT_DIRECTORY": modelB_s3_uri,
    },
}

### Create Inference containter definition for Model C

In [ ]:
pt_ecr_image_uriC = retrieve(
    framework='pytorch',
    region=region,
    version='1.8.1',
    py_version='py3',
    instance_type = deploy_instance_type,
    accelerator_type=None,
    image_scope='inference'
)

pt_containerC = {
    "ContainerHostname": "pytorch-kornlp-kobart",
    "Image": pt_ecr_image_uriC,
    "ModelDataUrl": modelC_s3_uri,
    "Environment": {
        "SAGEMAKER_PROGRAM": "inference_kobart.py",
        "SAGEMAKER_SUBMIT_DIRECTORY": modelC_s3_uri,
    },
}

### Create a SageMaker Model

`create_model` API를 호출하여 위 코드 셀에서 생성한 다중 컨테이너의 정의를 포함하는 모델을 생성합니다. 기존 엔드포인트와의 차이점은 `Containers` 매개 변수를 설정하고 `InferenceExecutionConfig` 매개변수의 Mode를 `Direct`나 `Serial`로 설정하는 것입니다. 기본 모드는 `Serial`이지만, 각 컨테이너를 직접 호출하려면 `Direct`로 설정해야 합니다. 자세한 내용은 [멀티 컨테이너 엔드포인트 배포](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html)를 확인하세요.

In [ ]:
model_name = f"KorNLPMultiContainer-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"

create_model_response = sm_client.create_model(
    ModelName=model_name,
    Containers=[pt_containerA, pt_containerB, pt_containerC],
    InferenceExecutionConfig={"Mode": "Direct"},
    ExecutionRoleArn=role,
)
print(f"Created Model: {create_model_response['ModelArn']}")

### Create Endpoint Configuration

In [ ]:
endpoint_config_name = f"KorNLPMultiContainerEndpointConfig-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "prod",
            "ModelName": model_name,
            "InitialInstanceCount": 1,
            "InstanceType": deploy_instance_type,
        },
    ],
)
print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

### Create a SageMaker Multi-container endpoint

create_endpoint API로 멀티 컨테이너 엔드포인트를 생성합니다. 기존의 엔드포인트 생성 방법과 동일합니다.

In [ ]:
endpoint_name = f"KorNLPMultiContainerEndpoint-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(f"Creating Endpoint: {endpoint_response['EndpointArn']}")

`describe_endpoint` API를 사용하여 엔드포인트 생성 상태를 확인할 수 있습니다. 엔드포인트 생성은 약 5분에서 10분이 소요됩니다.

In [ ]:
waiter = boto3.client('sagemaker').get_waiter('endpoint_in_service')
print("Waiting for endpoint to create...")
waiter.wait(EndpointName=endpoint_name)
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
print(f"Endpoint Status: {resp['EndpointStatus']}")

### Direct Invocation for Model A

두 문장간의 유사도를 정량화하는 예시입니다.
- KorNLI and KorSTS: https://github.com/kakaobrain/KorNLUDatasets

In [ ]:
modelA_sample_path = 'samples/nsmc.txt'
!cat $modelA_sample_path
with open(modelA_sample_path, mode='rb') as file:
    modelA_input_data = file.read()  

modelA_response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    TargetContainerHostname="pytorch-kornlp-nsmc",
    Body=modelA_input_data
)

modelA_outputs = modelA_response['Body'].read().decode()
print()
print_outputs(modelA_outputs)

### Direct Invocation for Model B

두 문장간의 유사도를 정량화하는 예시입니다.
- KorNLI and KorSTS: https://github.com/kakaobrain/KorNLUDatasets

In [ ]:
modelB_sample_path = 'samples/korsts.txt'
!cat $modelB_sample_path
with open(modelB_sample_path, mode='rb') as file:
    modelB_input_data = file.read()    
    
modelB_response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    TargetContainerHostname="pytorch-kornlp-korsts",
    Body=modelB_input_data
)

modelB_outputs = modelB_response['Body'].read().decode()
print()
print_outputs(modelB_outputs)

### Direct Invocation for Model C

문서 내용(예: 뉴스 기사)을 요약하는 예시입니다.

- KoBART: https://github.com/SKT-AI/KoBART
- KoBART Summarization: https://github.com/seujung/KoBART-summarization

In [ ]:
modelC_sample_path = 'samples/kobart.txt'
!cat $modelC_sample_path
with open(modelC_sample_path, mode='rb') as file:
    modelC_input_data = file.read()    
    
modelC_response = sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    TargetContainerHostname="pytorch-kornlp-kobart",
    Body=modelC_input_data
)

modelC_outputs = modelC_response['Body'].read().decode()
print()
print_outputs(modelC_outputs)

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)